<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_7_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,Discoloration of skin [blotchy] on the neck
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-7.test.csv",header=None)
test.head()

,0,1
0,3,joints poping and cracking
1,8,Arthritis
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'fatigue',
 'rt',
 'loss',
 'by',
 'limb',
 'cramp',
 'weakness',
 'sodium',
 'at',
 'disease',
 'stomach',
 'that',
 'as',
 'or',
 'back',
 'depression',
 '-',
 'patients',
 'from',
 'i',
 'not',
 'this',
 'cramps',
 'gas',
 'joint',
 'upper',
 'memory',
 'leg',
 'mg',
 'drug',
 'you',
 'all',
 'legs',
 'pains',
 'headache',
 'group',
 'knee',
 'be',
 'are',
 'shoulder',
 'study',
 'unable',
 'gastrointestinal',
 'arthralgia',
 'excessive',
 'treatment',
 'neck',
 'foot',
 'arthritis',
 'it',
 'aches',
 'hip',
 'after',
 'an',
 'amp',
 'have',
 'voltaren',
 'p',
 'numbness',
 'like',
 'no',
 'arthrotec',
 'insomnia',
 'increased',
 'release',
 '/',
 'nausea',
 'time',
 'gluten',
 'bleeding',
 'can',
 'both',
 'swelling',

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.859032,3.502401,0.440817,1:37:37


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.507537,3.441150,0.446116,1:35:36


In [25]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold7_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.943352,3.727366,0.322904,03:06


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.786284,3.039060,0.474343,03:22


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.244147,2.607785,0.534418,07:31


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.894391,2.173949,0.624531,07:38
1,1.603365,1.997438,0.663329,08:22


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.485564,1.845883,0.677096,08:49
1,1.314104,1.729396,0.693367,08:46


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.243096,1.636545,0.704631,08:13
1,1.009284,1.529205,0.715895,08:03


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.971201,1.454730,0.730914,08:08
1,0.856641,1.400212,0.749687,08:09


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.762619,1.369884,0.764706,08:08
1,0.718155,1.300884,0.768461,07:51


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.652833,1.273963,0.774718,08:21
1,0.638731,1.224849,0.783479,07:53
2,0.570994,1.193016,0.792240,07:45
3,0.468389,1.185417,0.795995,07:54


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.489273,1.171926,0.792240,08:00
1,0.501249,1.121205,0.799750,08:03
2,0.442248,1.141381,0.803504,08:09
3,0.383131,1.130090,0.803504,08:23


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.360248,1.210968,0.795995,07:58
1,0.400624,1.158398,0.799750,08:34
2,0.332689,1.097132,0.818523,08:47
3,0.313365,1.064523,0.817272,07:49


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.308886,1.138272,0.812265,08:33
1,0.348174,1.082146,0.812265,07:33
2,0.287409,1.094683,0.813517,08:09
3,0.236213,1.100038,0.821026,07:39


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.270484,1.098969,0.824781,07:40
1,0.265096,1.124094,0.826033,08:04
2,0.246873,1.077154,0.823529,08:02
3,0.221500,1.085955,0.822278,07:52


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.226028,1.135247,0.818523,08:47
1,0.264740,1.202442,0.811014,07:53
2,0.220981,1.135040,0.824781,08:15
3,0.199972,1.132925,0.833542,07:56


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.184367,1.171489,0.826033,07:58
1,0.237456,1.206740,0.827284,07:57
2,0.209602,1.101765,0.827284,08:29
3,0.190047,1.147073,0.826033,08:29


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.200744,1.220463,0.818523,08:56
1,0.209514,1.177041,0.824781,07:54


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.221745,1.206998,0.824781,08:40
1,0.192535,1.174669,0.826033,09:03


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.195872,1.225886,0.812265,08:04


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold7')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[326, 8, 8, 8, 8, 8, 8, 9, 98, 13, 536, 27, 27, 27, 503, 705, 30, 31, 584, 32, 741, 37, 37, 37, 741, 289, 448, 495, 49, 49, 49, 96, 292, 49, 49, 49, 49, 216, 49, 49, 49, 49, 49, 904, 53, 53, 53, 53, 58, 309, 113, 60, 61, 61, 62, 141, 71, 76, 76, 76, 76, 76, 79, 79, 79, 195, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 300, 195, 268, 88, 89, 89, 91, 91, 92, 92, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 49, 100, 100, 773, 100, 49, 100, 100, 100, 100, 100, 414, 8, 108, 111, 111, 111, 111, 114, 323, 75, 469, 124, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 127, 127, 624, 132, 139, 142, 148, 153, 160, 162, 166, 166, 129, 614, 969, 181, 289, 182, 182, 182, 182, 183, 183, 183, 183, 58, 187, 187, 187, 189, 354, 189, 289, 190, 190, 190, 190, 190, 190, 190, 190, 190, 336, 427, 197, 209, 197, 197, 197, 200, 203, 260, 205, 205, 210, 21

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

667
0.7702078521939953
